In [1]:
import requests
from bs4 import BeautifulSoup
from fake_useragent import UserAgent
from google.cloud import language_v1

from datetime import datetime
import re
from collections import defaultdict
import json
import pprint

from crawler_model import Ptt

ua = UserAgent()
fakeua = ua.random

In [ ]:
# Read
with open('phone_catalog.json', 'r') as fp:
    phone_dict = json.load(fp)

# phone_list

In [ ]:
def get_phones():
    url = "https://en.wikipedia.org/wiki/List_of_Android_smartphones"
    resp = requests.get(url)
    soup = BeautifulSoup(resp.text, "html.parser")

    tables = soup.find_all("table", class_="wikitable")
    rows = []
    for i in tables:
        rows.extend(i.select("tbody tr th"))

    phones = [a.text.partition("(")[0].strip() for a in rows]
    phones = [ p for p in phones if len(p.split()) > 1]

    brands = set()
    for p in phones:
        brands.add(p.split(" ")[0])
    brands = list(brands)
    brands = sorted(brands)
    for b in range(len(brands)):
        if brands[b] == "Asus":
            brands = brands[b:]
            break

    phone_list = []
    for b in brands:
        if b == "Samsung":
            for p in phones:
                if p.split(" ")[0] == b:
                    if "/" in p:
                        p_lst = p.split("/")
                        for series in p_lst[1:]:
                            sub_p = p_lst[0].replace("5G", "").strip()
                            if series == "+":
                                phone = sub_p + series
                            else:
                                phone = sub_p + " " + series
                            if len(phone) > 0:
                                phone_list.append(phone)
                        phone_list.append(sub_p)
                    else:
                        phone = p.replace("5G", "").strip()
                        if len(phone) > 0:
                            phone_list.append(phone)
        elif b == "POCO":
            for p in phones:
                if p.split(" ")[0] == b:
                    phone = p.replace("5G", "").strip()
                    if len(phone) > 0:
                        phone_list.append(phone)

        elif b == "Pixel":
            for p in phones:
                if p.split(" ")[0] == b:
                    phone = p.replace("5G", "").strip()
                    if len(phone) > 0:
                        phone_list.append(phone)
        else:
            for p in phones:
                if p.split(" ")[0] == b:
                    if "/" in p:
                        p_lst = p.split("/")
                        for series in p_lst[1:]:
                            sub_p = p_lst[0].replace("5G", "").strip()
                            if series == "+":
                                phone = sub_p + series
                            else:
                                phone = sub_p + " " + series
                            if len(phone) > 0:
                                phone_list.append(phone)
                        phone_list.append(sub_p)
                    else:
                        phone = p.replace("5G", "").strip()
                        if len(phone) > 0:
                            phone_list.append(phone)
                        phone_list.append(phone)
    phone_list.append("Asus ROG Phone 2")
    extend = ["note20ultra", "s20ultra", "s21ultra", "s20+", "s21+", "s20", "s21"]
    for s in extend:
        phone_list.append(s)

    url = "https://www.theiphonewiki.com/wiki/List_of_iPhones"
    resp = requests.get(url)
    soup = BeautifulSoup(resp.text, "html.parser")

    table = soup.find("div", id="toc")
    iphones = []
    for span in table.select("ul li span"):
        if "iPhone" in str(span):
            if ("SE" in str(span)) and ("1" in str(span)):
                iphones.append("iPhone SE")
            elif ("SE" in str(span)) and ("2" in str(span)):
                iphones.append("iPhone SE2")
            else:
                iphones.append(span.text)
    iphones.remove("iPhone")
    phone_list.extend(iphones)
    phone_set = set()
    for p in phone_list:
        phone_set.add(p)
    phone_list = sorted(list(phone_set))
    phone_list.remove("Asus ROG Phone II")
    return phone_list

# crawl PTT

In [ ]:
def get_last_page(url):
    ua = UserAgent()
    fakeua = ua.random
    headers = {"Origin": "https://www.ptt.cc",
            "Referer": "https://fonts.googleapis.com/",
            "sec-ch-ua-mobile": "?0",
            "User-Agent": fakeua}
    resp = requests.get(url, headers=headers)
    soup = BeautifulSoup(resp.text, "html.parser")
    previous = soup.find_all("a", class_="btn wide")[0]["href"]
    previous_page_num = int(re.findall(r"\d+", previous)[0])
    return previous_page_num
last_page = get_last_page("https://www.ptt.cc/bbs/MobileComm/search?page=1&q=%E5%BF%83%E5%BE%97")
last_page

In [ ]:
def sample_analyze_sentiment(text_content):
    """
    Analyzing Sentiment in a String

    Args:
      text_content The text content to analyze
    """
    os.environ["GOOGLE_APPLICATION_CREDENTIALS"]= os.path.join(os.path.expanduser("~"), "smartphone-smartprice-key.json")

    post = {}
    client = language_v1.LanguageServiceClient()

    # text_content = 'I am so happy and joyful.'

    # Available types: PLAIN_TEXT, HTML
    type_ = language_v1.Document.Type.PLAIN_TEXT

    # Optional. If not specified, the language is automatically detected.
    # For list of supported languages:
    # https://cloud.google.com/natural-language/docs/languages
    document = {"content": text_content, "type_": type_}

    # Available values: NONE, UTF8, UTF16, UTF32
    encoding_type = language_v1.EncodingType.UTF8

    response = client.analyze_sentiment(request = {'document': document, 'encoding_type': encoding_type})
    # Get overall sentiment of the input document
    doc = {}
    doc["score"] = response.document_sentiment.score
    
    doc["magnitude"] = response.document_sentiment.magnitude

    # Get sentiment for all sentences in the document
    sentences = []
    for sentence in response.sentences:
        # if abs(sentence.sentiment.score) >= 0.5 and sentence.sentiment.magnitude >= 0.5:
        d = {}
        d["content"] = sentence.text.content.replace(".", "").replace("。", "")
        d["score"] = sentence.sentiment.score
        d["magnitude"] = sentence.sentiment.magnitude
        sentences.append(d)
    
    return doc, sentences

def crawl_comm(links):
    for link in links:
        url = "https://www.ptt.cc" + link[1]
        # GET request from url and parse via BeautifulSoup
        ua = UserAgent()
        fakeua = ua.random
        headers = {"Origin": "https://www.ptt.cc",
                "Referer": "https://fonts.googleapis.com/",
                "sec-ch-ua-mobile": "?0",
                "User-Agent": fakeua}
        resp = requests.get(url, headers=headers)
        if "200" in str(resp):
            soup = BeautifulSoup(resp.text, "html.parser")
            main = soup.find("div", id="main-content")
            # keywords = ["優點", "缺點", "照相", "電池", "續航"]
            if "※ 引述" in str(main):   
                body = str(main).split('※ 引述')[0].split('</span></div>')[-1].replace(" ","").replace("\n", "。")
            else:
                body = str(main).split('--')[0].split('</span></div>')[-1].replace(" ","").replace("\n", "。")
        doc, sentences = sample_analyze_sentiment(body)
        d = {"title": link[0], "link": link[1], "doc": doc, "sentences": sentences}
        print(d)


In [ ]:
raw_data = []
data = []
error_links = []
for page in range(1, ):

    ua = UserAgent()
    fakeua = ua.random
    headers = {"Origin": "https://www.ptt.cc",
            "Referer": "https://fonts.googleapis.com/",
            "sec-ch-ua-mobile": "?0",
            "User-Agent": fakeua}

    url = "https://www.ptt.cc/bbs/MobileComm/search?page={}&q=%E5%BF%83%E5%BE%97".format(page)
    print(url)
    phone_list = get_phones()
    resp = requests.get(url, headers=headers)
    index_soup = BeautifulSoup(resp.text, "html.parser")
    keywords = ["換", "vs", "v.s", "vs.", "v.s."]
    title = index_soup.findAll("div", class_="title")

    links = []
    for t in title:
        if any(keyword in t.text for keyword in keywords):
            pass
        else:
            for p in phone_list:
                if p.replace(" ", "").lower() in t.text.replace(" ", "").lower():
                    print(p, t)
                    links.append( (t, t.find("a")["href"]) )
                    break
                    break
        

    crawl_comm(links)

In [ ]:
print(links)

In [ ]:
import pprint
print(len(data))
data[0]

# Sentiment

In [ ]:
from sentifish import Sentiment
text='''2020/02自美國雅馬遜購入
昨天發現電池膨脹了
爬了一下板上資訊
應該可以更換保固區域後以延長保固1年為由
更換（整）新機

但也看到線上服務看得是一種運
不是大好就是大壞
不巧小弟抽到壞的那邊
客服堅持不是官網買的無法處理
也沒有換保固區域的處理
另外也說電池膨脹不在延長保固之範圍
（但電量消耗大是可以的）

總之就是叫我自己去找雅馬遜處理

我想這樣的客服
我大概不會再買pixel了吧'''
obj=Sentiment(text)
polarity = obj.analyze( )
polarity

In [4]:
headers = {
    'user-agent': fakeua,
    'x-api-source': 'pc',
    'referer': "https://shopee.tw/product/354450050/8754002931"
    }
r = requests.get("https://shopee.tw/product/354450050/8754002931", headers=headers)
print(r)
soup = BeautifulSoup(r.text, "html.parser")
print(soup.prettfy())

NameError: name 'headers' is not defined